In [1]:
import math
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import cartopy.crs as ccrs
import cartopy.crs as crs
import cartopy.feature as cfeature
import h5py  
from astropy.time import Time
import os
import glob
from osgeo import gdal
import scipy.ndimage
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import csv

In [2]:
# all ATL08 points
all_points = pd.read_csv("D:\\masters_data\\output\\csv\\all_points.csv", header=None)
all_points.columns= ["latitude", "longitude", "mean_canopy", "max_canopy", "min_canopy", "median_canopy", "canopy_openness" ]

In [3]:
all_points

,latitude,longitude,mean_canopy,max_canopy,min_canopy,median_canopy,canopy_openness
0,10.808227,18.664986,4.863425,10.081665,0.511902,5.097809,3.428745
1,10.807324,18.664894,5.473944,10.648346,0.654572,5.853699,2.871783
2,10.806422,18.664801,4.802375,10.901764,0.598816,4.158936,3.598693
3,10.523349,18.636093,4.987176,9.200226,0.524872,4.552078,2.612066
4,3.786884,17.957226,25.025993,45.157928,6.172729,24.641403,10.615867
...,...,...,...,...,...,...,...
1215523,10.284673,27.424377,7.541678,13.522736,0.544708,7.244217,2.808851
1215524,10.285575,27.424288,4.563888,8.976044,0.692505,4.595215,2.192664
1215525,10.286477,27.424196,6.995975,13.343109,0.824402,7.031952,2.625959
1215526,10.287379,27.424105,7.050766,12.212524,0.525848,6.632889,2.707764


In [4]:
latitude = all_points['latitude'][:]
longitude = all_points['longitude'][:]
mean_height = all_points["mean_canopy"][:]
max_height = all_points["max_canopy"][:]
min_height = all_points["min_canopy"][:]
median_canopy = all_points["median_canopy"][:]
canopy_openness = all_points["canopy_openness"][:]

In [5]:
# constants for conversion lat lon -> tile H,V and r,c
R = 6371007.181	    # the radius of the idealizes sphere representing the Earth
T = 1111950	        # the height and width of each MODIS tile in the projection plane
Xmin = -20015109    # the western limit of the projection plane
Ymax = 10007555	    # the northern limit of the projection plane
w = T/2400.0	    # the actual size of a '500m' MODIS sinusoidal grid cell. (would be /1200 for a 1km grid)

In [6]:
H_list = []
V_list = []
row_list = []
col_list = []

for jj in range(len(latitude)):
    Rlat = math.radians(latitude[jj])
    Rlon = math.radians(longitude[jj])

    x = R*Rlon*(math.cos(Rlat))
    y = R*Rlat

    H = math.floor((x - Xmin)/T)
    V = math.floor((Ymax - y)/T)
    
    row = math.floor(((Ymax - y)%T)/w -0.5)
    col = math.floor(((x - Xmin)%T)/w -0.5)

    H_list.append(H)
    V_list.append(V)
    row_list.append(row)
    col_list.append(col)

In [7]:
data = pd.DataFrame({"latitude":latitude,"longitude": longitude, 
                     "H": H_list, "V": V_list, "row": row_list, "col": col_list, 
                     "mean_canopy": mean_height, "min_height": min_height, 
                     "max_canopy": max_height, "median_canopy": median_canopy,
                     "canopy_openness": canopy_openness})

In [8]:
data.drop(data[data['V'] == 9].index, inplace = True)
data.drop(data[data['row'] == -1].index, inplace = True)
data.drop(data[data['col'] == -1].index, inplace = True)

data['H'] = data['H'].astype(int)
data['V'] = data['V'].astype(int)
data = data.reset_index(drop=True)

*** LOAD DATA ***

In [9]:
# VI
H19V07_VI = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H19V07_VI_array.csv", header=None)
H19V08_VI = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H19V08_VI_array.csv", header=None)
H20V07_VI = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H20V07_VI_array.csv", header=None)
H20V08_VI = pd.read_csv("D:\\masters_data\\output\\csv\\NDVI\\Total\\H20V08_VI_array.csv", header=None)

In [10]:
# Rainfall
H19V07_Rain = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H19V07_Rainfall.csv", header=None)
H19V08_Rain = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H19V08_Rainfall.csv", header=None)
H20V07_Rain = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H20V07_Rainfall.csv", header=None)
H20V08_Rain = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H20V08_Rainfall.csv", header=None)

In [11]:
#LC
H19V07_LC = pd.read_csv("D:\\masters_data\\MODIS_LC\\LC_Data\\MCD12Q1.A2008001.h19v07.006.2018145230513.hdf", header=None)
H19V08_LC = pd.read_csv("D:\\masters_data\\MODIS_LC\\LC_Data\\MCD12Q1.A2008001.h19v08.006.2018145230512.hdf", header=None)
H20V07_LC = pd.read_csv("D:\\masters_data\\MODIS_LC\\LC_Data\\MCD12Q1.A2008001.h20v07.006.2018145230622.hdf", header=None)
H20V08_LC = pd.read_csv("D:\\masters_data\\MODIS_LC\\LC_Data\\MCD12Q1.A2008001.h20v08.006.2018145230633.hdf", header=None)

In [12]:
# BA
# H19V07
H19V07_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\first_BA_500-1500_H19V07.csv", header=None)
H19V07_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\second_BA_500-1500_H19V07.csv", header=None)
H19V07_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\first_BA_sub500_H19V07.csv", header=None)
H19V07_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\second_BA_sub500_H19V07.csv", header=None)
H19V07_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\first_BA_above_1500_H19V07.csv", header=None)
H19V07_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V07\\second_BA_above_1500_H19V07.csv", header=None)

# H19V08
H19V08_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_BA_500-1500_H19V08.csv", header=None)
H19V08_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_500-1500_H19V08.csv", header=None)
H19V08_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_BA_sub500_H19V08.csv", header=None)
H19V08_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_sub500_H19V08.csv", header=None)
H19V08_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\first_above_1500_H19V08.csv", header=None)
H19V08_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H19_V08\\second_BA_above_1500_H19V08.csv", header=None)

# H20V07
H20V07_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\first_BA_500-1500_H20V07.csv", header=None)
H20V07_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\second_BA_500-1500_H20V07.csv", header=None)
H20V07_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\first_BA_sub500_H20V07.csv", header=None)
H20V07_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\second_BA_sub500_H20V07.csv", header=None)
H20V07_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\first_above_1500_H20V07.csv", header=None)
H20V07_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V07\\second_BA_above_1500_H20V07.csv", header=None)
Yearly_H20V07_Rainfall = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H20V07_Rainfall.csv", header=None)

# H20V08
H20V08_First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_500-1500_H20V08.csv", header=None)
H20V08_Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_500-1500_H20V08.csv", header=None)
H20V08_First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_sub500_H20V08.csv", header=None)
H20V08_Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_sub500_H20V08.csv", header=None)
H20V08_First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_above1500_H20V08.csv", header=None)
H20V08_Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_above1500_H20V08.csv", header=None)
Yearly_H20V08_Rainfall = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\total_rainfall\\Yearly_H20V08_Rainfall.csv", header=None)

In [13]:
# creating BA masks - maybe redo this in a more sensible way?

# H19V07
H19V07_first_BA = H19V07_First_BA_500_1500 + H19V07_First_BA_sub500 + H19V07_First_BA_above_1500
H19V07_second_BA = H19V07_Second_BA_500_1500 + H19V07_Second_BA_sub500 + H19V07_Second_BA_above_1500
H19V07_total_BA = H19V07_first_BA + H19V07_second_BA 

# H19V08
H19V08_first_BA = H19V08_First_BA_500_1500 + H19V08_First_BA_sub500 + H19V08_First_BA_above_1500
H19V08_second_BA = H19V08_Second_BA_500_1500 + H19V08_Second_BA_sub500 + H19V08_Second_BA_above_1500
H19V08_total_BA = H19V08_first_BA + H19V08_second_BA

# H20V07
H20V07_first_BA = H20V07_First_BA_500_1500 + H20V07_First_BA_sub500 + H20V07_First_BA_above_1500
H20V07_second_BA = H20V07_Second_BA_500_1500 + H20V07_Second_BA_sub500 + H20V07_Second_BA_above_1500
H20V07_total_BA = H20V07_first_BA + H20V07_second_BA

# H20V08
H20V08_first_BA = H20V08_First_BA_500_1500 + H20V08_First_BA_sub500 + H20V08_First_BA_above_1500
H20V08_second_BA = H20V08_Second_BA_500_1500 + H20V08_Second_BA_sub500 + H20V08_Second_BA_above_1500
H20V08_total_BA = H20V08_first_BA + H20V08_second_BA

In [14]:
# create columns
data['VI'] = -999.99
data['Total_rainfall'] = -999.99
data['LC'] = -9
data['First_burns'] = -99
data['Last_burns'] = -99
data['Total_burns'] = -99

In [15]:
def BurnCategory(Firstburn, Secondburn):
    if Firstburn < 1:
        if Secondburn < 1:
            category = "low_low"
        elif Secondburn >= 6:
            category = "low_high"
        else:
            category = "low_med"
            
    elif Firstburn >= 6:
        if Secondburn < 1:
            category = "high_low"
        elif Secondburn >= 6:
            category = "high_high"
        else:
            category = "high_med"
            
    else:
        if Secondburn < 1:
            category = "med_low"
        elif Secondburn >= 6:
            category = "med_high"
        else:
            category = "med_med"
    return category

In [16]:
def RainfallBand_500(Total):
    if Total < 500:
        category = "sub_500"
    elif Total >= 1500:
        category = "above_1500"
    else:
        category = "500_1500"
        return category

In [17]:
def RainfallBand_200(Total):
    if Total < 200:
        category = 100
    elif 200 < Total <  400:
        category = "300"
    elif 400 < Total <  600:
        category = "500"
    elif 600 < Total <  800:
        category = "700"
    elif 800 < Total <  1000:
        category = "900"
    elif 1000 < Total < 1200:
        category = "1100"
    elif 1200 < Total < 1400:
        category = "1300"
    elif 1400 < Total < 1600:
        category = "1500"
    elif 1600 < Total < 1800:
        category = "1700"
    elif 1800 < Total < 2000:
        category = "1900"
    elif 2000 < Total < 2200:
        category = "2100"
    elif 2200 < Total < 2400:
        category = "2300"
    elif 2400 < Total < 2600:
        category = "2500"
    return category

In [ ]:
# NOTE: RAINFALL ROW/COL WAS CALCULATED THE WRONG WAY ROUND. HERE I AM INVERSING IT AGAIN 
# SO ITS FIXED IN THE DATAFRAME. WHEN RAINFALL IS FIXED I WILL HAVE TO REVERSE THIS HERE.
for i in range(len(data)):
    H = data['H'][i]
    V = data['V'][i]
    row = data['row'][i]
    col = data['col'][i]
    
    if (H == 19) & (V == 7):
        data['VI'][i] = H19V07_VI[col][row]
        data['Total_rainfall'][i] = H19V07_Rain[row][col]
        data['LC'][i] = H19V07_LC[col][row]
        data['First_burns'][i] = H19V07_first_BA[col][row]
        data['Last_burns'][i] = H19V07_second_BA[col][row]
        data['Total_burns'][i] = H19V07_total_BA[col][row]
        
    elif (H == 19) & (V == 8):
        data['VI'][i] = H19V08_VI[col][row]
        data['Total_rainfall'][i] = H19V08_Rain[row][col]
        data['LC'][i] = H19V08_LC[col][row]
        data['First_burns'][i] = H19V08_first_BA[col][row]
        data['Last_burns'][i] = H19V08_second_BA[col][row]
        data['Total_burns'][i] = H19V08_total_BA[col][row]
        
    elif (H == 20) & (V == 7):
        data['VI'][i] = H20V07_VI[col][row]
        data['Total_rainfall'][i] = H20V07_Rain[row][col]
        data['LC'][i] = H20V07_LC[col][row]
        data['First_burns'][i] = H20V07_first_BA[col][row]
        data['Last_burns'][i] = H20V07_second_BA[col][row]
        data['Total_burns'][i] = H20V07_total_BA[col][row]
        
    elif (H == 20) & (V == 8):
        data['VI'][i] = H20V08_VI[col][row]
        data['Total_rainfall'][i] = H20V08_Rain[row][col]
        data['LC'][i] = H20V08_LC[col][row]
        data['First_burns'][i] = H20V08_first_BA[col][row]
        data['Last_burns'][i] = H20V08_second_BA[col][row]
        data['Total_burns'][i] = H20V08_total_BA[col][row]

data['Tile'] = "H" + data['H'].astype(str) + "V0" + data['V'].astype(str)

In [ ]:
data['BA_category'] = "none"

In [ ]:
for i in range(len(data)):
    data['BA_category'][i] = BurnCategory(data['First_burns'][i], data['Last_burns'][i])

In [ ]:
np.savetxt("D:\\masters_data\\output\\csv\\Analysis\\data2.csv", data, delimiter=",", fmt='%s')

In [27]:
data['BA_category'].unique()

array(['highigh', 'lowlow', 'lowmed', 'medmed', 'medlow', 'medhigh',
       'lowhigh', 'highmed', 'highlow'], dtype=object)

In [28]:
# Do I want canopy spread in here too?
data['canopy_spread'] = (data['max_canopy'] - data['min_height']) /  data['mean_canopy']

In [ ]:
# plotting the data without the correcting for anomalies

variable = 'VI' #what we want the colour to represent

## variable min and max (I could probably make this automatic) 
ticks=np.arange(-20,100,5)

#size of image
plt.figure(figsize=(8,8), dpi= 90)

#coordinates / projection
ax = plt.axes(projection=crs.PlateCarree())
lat1, lon1, lat2, lon2 = -20, 35, -20, 50
ax.set_extent([lat1, lon1, lat2, lon2], crs=crs.PlateCarree())

#background and design
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.stock_img()
ax.add_feature(cfeature.BORDERS)
ax.set_title('{} height'.format(variable), fontsize = 20, color = 'gray')

#adding variable onto map (and colour bar)
plt.scatter(data['longitude'][::], data['latitude'][::],c=data[variable][::], cmap='viridis',transform=ccrs.PlateCarree())
plt.colorbar(label=variable, shrink=0.5, ticks=ticks,extend='both')

In [ ]:
# plotting the data without the correcting for anomalies

variable = 'VI' #what we want the colour to represent

## variable min and max (I could probably make this automatic) 
ticks=np.arange(-20,100,5)

#size of image
plt.figure(figsize=(8,8), dpi= 90)

#coordinates / projection
ax = plt.axes(projection=crs.PlateCarree())
lat1, lon1, lat2, lon2 = -20, 35, -20, 50
ax.set_extent([lat1, lon1, lat2, lon2], crs=crs.PlateCarree())

#background and design
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.stock_img()
ax.add_feature(cfeature.BORDERS)
ax.set_title('{} height'.format(variable), fontsize = 20, color = 'gray')

#adding variable onto map (and colour bar)
plt.scatter(data5['longitude'][::], data5['latitude'][::],c=data5[variable][::], cmap='viridis',transform=ccrs.PlateCarree())
plt.colorbar(label=variable, shrink=0.5, ticks=ticks,extend='both')

In [ ]:
# just H20 I need to fix

In [ ]:
# plotting the data without the correcting for anomalies

variable = 'Total_rainfall' #what we want the colour to represent

## variable min and max (I could probably make this automatic) 
ticks=np.arange(-20,100,5)

#size of image
plt.figure(figsize=(8,8), dpi= 90)

#coordinates / projection
ax = plt.axes(projection=crs.PlateCarree())
lat1, lon1, lat2, lon2 = -20, 35, -20, 50
ax.set_extent([lat1, lon1, lat2, lon2], crs=crs.PlateCarree())

#background and design
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.stock_img()
ax.add_feature(cfeature.BORDERS)
ax.set_title('{} height'.format(variable), fontsize = 20, color = 'gray')

#adding variable onto map (and colour bar)
plt.scatter(data['longitude'][::], data['latitude'][::],c=data[variable][::], cmap='viridis',transform=ccrs.PlateCarree())
plt.colorbar(label=variable, shrink=0.5, ticks=ticks,extend='both')

In [ ]:
H19V07_first_BA

In [ ]:
data

In [ ]:
data['Rainfall_band200'] = -999
data['Rainfall_band500'] = "None"

def RainfallBand_200(Total):
    if Total < 200:
        category = 100
    elif 200 < Total <  400:
        category = 300
    elif 400 < Total <  600:
        category = 500
    elif 600 < Total <  800:
        category = 700
    elif 800 < Total <  1000:
        category = 900
    elif 1000 < Total < 1200:
        category = 1100
    elif 1200 < Total < 1400:
        category = 1300
    elif 1400 < Total < 1600:
        category = 1500
    elif 1600 < Total < 1800:
        category = 1700
    elif 1800 < Total < 2000:
        category = 1900
    elif 2000 < Total < 2200:
        category = 2100
    elif 2200 < Total < 2400:
        category = 2300
    return category

def RainfallBand_500(Total):
    if Total < 500:
        category = "sub_500"
    elif Total >= 1500:
        category = "above_1500"
    else:
        category = "500_1500"
        return category

for i in range(len(data)):
    data['Rainfall_band200'][i] = RainfallBand_200(data['Yearly_rainfall'][i])
    data['Rainfall_band500'][i] = RainfallBand_500(data['Yearly_rainfall'][i])